# <center> Implementation of  [Lar vonn et. al. ](https://chemistry-europe.onlinelibrary.wiley.com/doi/full/10.1002/cssc.202000867) paper </center>

"The total current density is divided into two contributions: intercalation and SEI. $J_\mathrm{total} = J_\mathrm{int} + J_\mathrm{SEI}$


$\begin{equation}
   J_\mathrm{total} = J_\mathrm{int} + J_\mathrm{SEI}  \tag{3}
\end{equation}$

$\begin{equation}
    J_\mathrm{int} = 2 j_0 \sinh{ \left ( \frac{F}{2RT} \eta_{\mathrm{int}} \right ) },  \tag{9}
\end{equation}$

$\begin{equation}
    \frac{\text{d}c_s}{\text{d}t} = - \frac{a_n}{F} J_\mathrm{int}  \tag{12}
\end{equation}$

$\begin{equation}
    \frac{\text{d}L_\mathrm{SEI}}{\text{d}t} = - \frac{V_\mathrm{SEI}}{F} J_\mathrm{SEI}  \tag{22}
\end{equation}$


$\begin{equation}
     J_{\mathrm{SEI}}=-j_{\mathrm{SEI}, 0,0} e^{-\alpha_{\mathrm{SEI}} \tilde{\eta}_{\mathrm{SEI}}} \frac{1 \pm \frac{L_{\text {app }}}{L_{\text {mig }}}}{1 \pm \frac{L_{\text {app }}}{L_{\text {mig }}}+\frac{L_{\text {app }}}{L_{\text {diff }}}},  \tag{19}
 \end{equation}$
 

 $$
    L_{\text {diff }}=\frac{c_{\mathrm{Li}, 0} D F}{j_{\mathrm{SEl}, 0,0}} e^{-\left(1-a_{\mathrm{SE}}\right) \tilde{\eta}_{\mathrm{SEl}}},
$$

$$
    L_{\text {mig }}=\frac{2 R T \kappa_{\mathrm{Li}^{+}, \text {SEI }}}{F\left|J_{\text {int }}\right|},
 $$

$$
    \tilde{\eta}_{\mathrm{SEl}}=\frac{F}{R T}\left(\eta_{\mathrm{int}}+U_{0}+\mu_{\mathrm{Li}, 0} / F\right),
$$

$$
    \eta_\mathrm{int} = \phi_s -  U_0 - \mu_{\mathrm{Li^+, SEI}},
$$

$$
    U_{0}=\left(\frac{1.24-c_{\mathrm{s}} / c_{\mathrm{s}, \max }}{1.16}\right)^{2.92} \mathrm{~V}
$$


$$
    j_{0} = j_{0,0} \sqrt{\frac{c_s}{c_{s, max}}},
$$

We are trying two solve DAEs with (3,9, 12, 19, 22) for the variables $J_\mathrm{int}, J_\mathrm{SEI}, cs, L_\mathrm{SEI}, \eta_\mathrm{int}$ with approperiate initial conditions. 

In [55]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
# import time
# import scipy.sparse as sparse
# pybamm.__version__

Defining varaibles

In [56]:
from pybamm import exp
from pybamm import tanh

cs = pybamm.Variable("concentration [mol.m-3]")
# J_int = pybamm.Variable("Intercalation current density [A.m-2]")
J_SEI = pybamm.Variable("SEI interfacial density [A.m-2]")
L_SEI = pybamm.Variable("Thickness of SEI [m]")
eta_int = pybamm.Variable("Overpotential [V]")

Define Model Parameters

In [57]:

c_Li = pybamm.Parameter(
    'Lithium atom reference concentration [mol.m-3]')  # 1 Lar
alpha_SEI = pybamm.Parameter("alpha_SEI [-]")  # 0.22 Lar
j_SEI_0_0 = pybamm.Parameter(
    'Initial SEI reaction exchange current density [A.m-2]')  # 7.04e-5 Lar
T = pybamm.Parameter('Initial temperature [K]')  # 303 Lar
R = pybamm.Parameter('Ideal gas constant [J.K-1.mol-1]')  # 8.314462618 Lar
F = pybamm.Parameter("Faraday constant [C.mol-1]")  # 96485 Lar
cs_max = pybamm.Parameter(
    "maximum lithium ion concentration in electrode [mol.m-3]")  # 3.16 lar
v_SEI = pybamm.Parameter("Mean moler volume of SEI [m3.mol-1]")  # 1.078e-5 Lar
kappa_Li_SEI = pybamm.Parameter(
    "Lithium ion conductivity of the SEI [S.m-1]")  # 1e-8 Lar
mu_Li_0 = pybamm.Parameter(
    "Chemical reference potential of Li^0 [kJ.mol-1]")  # 17.4 Lar
L_app_0 = pybamm.Parameter("Reference apparent thickness  [nm]")  # 0.05 Lar
L_tun = pybamm.Parameter("typical tunning distance  [nm]")  # 2.05 Lar
U1 = pybamm.Parameter("Maximum Voltage during cycling [V]")  # 1.2 Lar
U2 = pybamm.Parameter("Minimum Voltage during cycling [V]")  # 0.01 Lar
j0_0 = pybamm.Parameter(
    "Butler-Volmer rate constant for intercalation [A.m-2]")  # 6.4e-7 Lar
D_Li = pybamm.Parameter(
    "Diffusion coefficient of Li atoms inside the SEI [m2.s-1]")  # 1 × 10−15 Lar
J_total = pybamm.Parameter("Current density [A.m-2]")  # 0.1 Lar
A_cb = pybamm.Parameter(
    "Surface area to volume ratio [m-1]")  # 86.8 × 10^6 Lar
L_SEI_0 = pybamm.Parameter("Initial thickness of SEI [m]")  # 2e-9 Lar
cs_0 = pybamm.Parameter("Initial concentration [mol.m-3]")  # 0.01 Lar


def U0(x):
    return pybamm.FunctionParameter(
        "open circuit voltage OCV [V]",
        {"Negative particle stoichiometry": x},
    )


def OCP(sto):
    u_eq = (
        ((1.24 - sto)/1.16)**2.92
    )
    return u_eq

Parameter values from Lars von Wedel 2020  https://doi.org/10.1016/j.jpowsour.2020.228923 suplementary material.

In [58]:
params = pybamm.ParameterValues(
    {'Lithium atom reference concentration [mol.m-3]': 1,
     "alpha_SEI [-]": 0.22,
     'Initial SEI reaction exchange current density [A.m-2]': 7.04e-5,
     'Initial temperature [K]': 303,
     'Ideal gas constant [J.K-1.mol-1]': 8.314462618,
     "Faraday constant [C.mol-1]": 96485,
     "maximum lithium ion concentration in electrode [mol.m-3]": 30006,
     "Mean moler volume of SEI [m3.mol-1]": 1.078e-5,
     "Lithium ion conductivity of the SEI [S.m-1]": 1e-8,
     "Chemical reference potential of Li^0 [kJ.mol-1]": 17.4e3,
     "Reference apparent thickness  [nm]": 0.05e-9,
     "typical tunning distance  [nm]": 2.05e-9,
     "Maximum Voltage during cycling [V]": 1.2,
     "Minimum Voltage during cycling [V]": 0.01,
     "Butler-Volmer rate constant for intercalation [A.m-2]": 6.4e-7,
     "Diffusion coefficient of Li atoms inside the SEI [m2.s-1]": 1e-15,
     "Surface area to volume ratio [m-1]": 86.8e5,
     "Initial thickness of SEI [m]": 2e-9,
     "open circuit voltage OCV [V]": OCP,
     "Applied current [A]": 0.1,
     "Current density [A.m-2]": 0.1,
     "Initial concentration [mol.m-3]": 0.01,

     }
)

In [59]:
model = pybamm.lithium_ion.BaseModel()

Defining variables of the model

In [60]:
model.variables = {
    "concentration [mol.m-3]": cs,
    # "Intercalation current density [A.m-2]": J_int,
    "SEI interfacial density [A.m-2]": J_SEI,
    "Thickness of SEI [m]": L_SEI,
    "Overpotential [V]": eta_int,

}

In [61]:
# from math import sinh

eta_bar_SEI = F/(R*T) * (eta_int + U0(cs/cs_max) + mu_Li_0/F)
L_diff = c_Li * D_Li * F / j_SEI_0_0 * pybamm.exp(-(1-alpha_SEI) * eta_bar_SEI)
L_mig = (2 * R * T * kappa_Li_SEI)/(F*pybamm.AbsoluteValue(J_total - J_SEI))
L_prime_app = L_SEI - L_tun
L_app = L_prime_app / 2 + ((L_prime_app/2)**10 + L_app_0**2)**10


model.algebraic = {
    J_SEI: J_SEI + j_SEI_0_0 * pybamm.exp(-alpha_SEI * eta_bar_SEI) * (1 + L_app/L_mig)/(1 + L_app/L_mig + L_app/L_diff),
    eta_int: (J_total - J_SEI) - 2*j0_0 * pybamm.sqrt(cs/cs_max) * pybamm.sinh(F/(2*R*T) * eta_int),
}

In [62]:

dcsdt = -A_cb*(J_total - J_SEI)/F
dL_SEIdt = - v_SEI * J_SEI / F


model.rhs = {cs: dcsdt, L_SEI: dL_SEIdt}

In [63]:
model.initial_conditions = {
    J_SEI: j_SEI_0_0,
    cs: 100,
    L_SEI: L_SEI_0,
    eta_int: 0.0,
}

In [64]:
params.process_model(model)
disc = pybamm.Discretisation()  # use the default discretisation
disc.process_model(model)

solver = pybamm.IDAKLUSolver()


# start_time = time.time()
sim = pybamm.Simulation(
    model, parameter_values=params, solver=solver)
solution = sim.solve([0, 100])

CasADi - 2023-11-15 17:24:18 WARNING("roots:g failed: Inf detected for output x, at (row 1, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-11-15 17:24:18 WARNING("roots:g failed: Inf detected for output x, at (row 1, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-11-15 17:24:18 WARNING("roots:g failed: Inf detected for output x, at (row 1, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-11-15 17:24:18 WARNING("roots:g failed: Inf detected for output x, at (row 1, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-11-15 17:24:18 WARNING("roots:g failed: Inf detected for output x, at (row 1, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-11-15 17:24:18 WARNING("roots:g failed: Inf detected for output x, at (row 1, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-11-15 17:24:18 WARNING("roots:g failed: Inf detected for output x, at (row 1, col 0).") [.../casadi/core/oracle_function.c

SolverError: idaklu solver failed

In [ ]:
solution["concentration [mol.m-3]"].entries

array([1000.        ,  990.91291209,  981.82582418,  972.73873628,
        963.65164837,  954.56456046,  945.47747255,  936.39038465,
        927.30329674,  918.21620883,  909.12912092,  900.04203302,
        890.95494511,  881.8678572 ,  872.78076929,  863.69368139,
        854.60659348,  845.51950557,  836.43241766,  827.34532976,
        818.25824185,  809.17115394,  800.08406603,  790.99697812,
        781.90989022,  772.82280231,  763.7357144 ,  754.64862649,
        745.56153859,  736.47445068,  727.38736277,  718.30027486,
        709.21318696,  700.12609905,  691.03901114,  681.95192323,
        672.86483533,  663.77774742,  654.69065951,  645.6035716 ,
        636.51648369,  627.42939579,  618.34230788,  609.25521997,
        600.16813206,  591.08104416,  581.99395625,  572.90686834,
        563.81978043,  554.73269253,  545.64560462,  536.55851671,
        527.4714288 ,  518.3843409 ,  509.29725299,  500.21016508,
        491.12307717,  482.03598927,  472.94890136,  463.86181